In [1]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment
import pywt

e:\Repositories\alcd\.venv\Lib\site-packages\pydub\utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
e:\Repositories\alcd\.venv\Lib\site-packages\pydub\utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
e:\Repositories\alcd\.venv\Lib\site-packages\pydub\utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
e:\Repositories\alcd\.venv\Lib\site-packages\pydub\utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


# Carregamento do arquivo

In [2]:
def load_mp3(filepath: pathlib.Path | str):
    audio = AudioSegment.from_mp3(filepath)
    y = np.array(audio.get_array_of_samples())
    return y, audio.frame_rate

In [7]:
file_name = "c_major_scale"
original_file = pathlib.Path(f"../resources/{file_name}.mp3")

In [8]:
audio, sampling_rate = load_mp3(original_file)

# Aplicação de ruído

In [23]:
noise_scale = 0.001 * 32768
noise = np.random.normal(0, noise_scale, audio.shape)
noisy_audio = audio + noise
noisy_audio = noisy_audio.astype(np.int16)

In [24]:
# Salvando para comparação futura
noisy_audio_path = pathlib.Path(f"../results/noisy_{file_name}.mp3")
AudioSegment(noisy_audio.tobytes(), frame_rate=sampling_rate, sample_width=noisy_audio.dtype.itemsize, channels=2).export(noisy_audio_path, format="mp3")

<_io.BufferedRandom name='..\\results\\noisy_c_major_scale.mp3'>

# Aplicação de algoritmos de _denoising_

In [25]:
n = 2
w = "haar"
noisy_audio_normalized = noisy_audio.astype(np.float64) / 32768.0

In [26]:
# Aplicação da Transformada Discreta da Wavelet (DWT)
coeffs = pywt.wavedec(noisy_audio_normalized, w, level=n)
# Definição de limiar para o encolhimento dos coeficientes
threshold = 0.04
coeffs_thresholded = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
# Reconstrução do sinal sem ruído
denoised_audio = pywt.waverec(coeffs_thresholded, w)
denoised_audio = np.clip(denoised_audio * 32768, -32768, 32767).astype(np.int16)
# Salvando para comparação futura
AudioSegment(denoised_audio.tobytes(), frame_rate=sampling_rate, sample_width=denoised_audio.dtype.itemsize, channels=2).export("denoised_scale.mp3", format="mp3")

<_io.BufferedRandom name='denoised_scale.mp3'>